# Incorporando Dados Existentes ao Apache Iceberg

Este notebook demonstra como migrar dados existentes em formato Parquet para tabelas Apache Iceberg. Esta é uma necessidade comum em projetos de modernização de data lakes, onde organizações já possuem dados em formatos tradicionais e desejam aproveitar os benefícios avançados do Iceberg.

## Contexto e Objetivos:

### Cenário Comum:
- Dados históricos armazenados em formato Parquet
- Necessidade de migrar para Iceberg sem perder dados
- Manter compatibilidade durante a transição
- Aproveitar recursos como ACID, time travel e schema evolution

### Benefícios da Migração:
- **ACID Transactions**: Garantia de consistência em operações
- **Schema Evolution**: Evolução de esquema sem reescrita
- **Time Travel**: Consultas históricas e rollbacks
- **Partition Evolution**: Mudança de estratégias de particionamento
- **Metadata Management**: Gerenciamento eficiente de metadados

### Estratégias de Migração:
1. **Migração Direta**: Leitura de Parquet e escrita em Iceberg
2. **Migração Incremental**: Processamento em lotes menores
3. **Migração com Transformação**: Aplicação de regras durante a migração

## Setup do Ambiente

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col

# Sessão Spark
# Para o Spark se estiver rodando
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder \
    .appName("IcebergRollbacks") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", "file:///home/tavares/warehouse") \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .getOrCreate()

print(f"Warehouse configurado para: {spark.conf.get('spark.sql.catalog.hadoop_catalog.warehouse')}")

Warehouse configurado para: file:///home/tavares/warehouse


## 1. Criação da Tabela Iceberg de Destino

Primeiro, criamos uma tabela Iceberg vazia com o mesmo esquema dos dados Parquet existentes. Esta tabela servirá como destino para a migração dos dados.

In [2]:
# Criar uma tabela Iceberg de destino
spark.sql("DROP TABLE IF EXISTS hadoop_catalog.default.vendas_iceberg")

spark.sql("""
    CREATE TABLE hadoop_catalog.default.vendas_iceberg (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
""")

DataFrame[]

## 2. Leitura e Migração dos Dados Parquet

Agora vamos ler os dados existentes em formato Parquet e migrá-los para a tabela Iceberg. Esta operação preserva todos os dados enquanto os converte para o formato Iceberg com seus benefícios adicionais.

In [3]:
# le parquet
df_parquet = spark.read.parquet("data/vendas_iceberg/")
# atualiza na tabela
df_parquet.writeTo("hadoop_catalog.default.vendas_iceberg").append()

## 3. Verificação dos Dados Migrados

Após a migração, é importante verificar se todos os dados foram transferidos corretamente e se mantêm sua integridade.

In [ ]:
#visualizamos os dados
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_iceberg order by id").show()

## 4. Verificação dos Snapshots Criados

Uma das vantagens imediatas da migração para Iceberg é o sistema de snapshots. Vamos verificar o snapshot criado durante a migração.

In [ ]:
# mostra snapshots
snapshots_df = spark.sql("SELECT * FROM hadoop_catalog.default.vendas_iceberg.snapshots")
snapshots_df.select("snapshot_id", "committed_at", "operation").show(truncate=False)